In [ ]:
#References:

#https://datasciencedojo.com/blog/scrape-twitter-data-using-snscrape/
#https://thetechrobo.ca/snscrape-docs/_autosummary/snscrape.modules.twitter.html

#Note1: Snscrape may no longer work due to changes in access to Twitter's data in June of 2023.
#Note2: I do not encourage anyone to scrape Twitter for any commercial purpose. 

In [32]:
import pandas as pd 
import numpy as np 
import snscrape.modules.twitter as sntwitter 
import datetime 
from tqdm.notebook import tqdm_notebook 
import seaborn as sns 
import matplotlib.pyplot as plt 

sns.set_theme(style="whitegrid") 

In [11]:
# Text: The query to be matched. (Optional) 
# Username: Specific username from twitter account. (Required) 
# Since: Start Date in this format yyyy-mm-dd. (Optional) 
# Until: End Date in this format yyyy-mm-dd. (Optional) 
# Count: Max number of tweets to retrieve. (Required) 
# Retweet: Include or Exclude Retweets. (Required) 
# Replies: Include or Exclude Replies. (Required) 

In [33]:
username = input('Enter specific username(s) from a twitter account without @ (or leave it blank by pressing enter): ') 
since = input('Enter startdate in this format yyyy-mm-dd (or leave it blank by pressing enter): ') 
until = input('Enter enddate in this format yyyy-mm-dd (or leave it blank by pressing enter): ') 
count = int(input('Enter max number of tweets or enter -1 to retrieve all possible tweets: ')) 
retweet = input('Exclude Retweets? (y/n): ') 
replies = input('Exclude Replies? (y/n): ') 

print("Username(s)        : ", username)
print("StartDate          : ", since)
print("EndDate            : ", until)
print("MaxTweets          : ", count)
print("Exclude Retweets?  : ", retweet)
print("Exclude Replies?   : ", replies)

Enter specific username(s) from a twitter account without @ (or leave it blank by pressing enter):  
Enter startdate in this format yyyy-mm-dd (or leave it blank by pressing enter):  1998-01-01
Enter enddate in this format yyyy-mm-dd (or leave it blank by pressing enter):  2008-07-31
Enter max number of tweets or enter -1 to retrieve all possible tweets:  -1
Exclude Retweets? (y/n):  y
Exclude Replies? (y/n):  y


Username(s)        :  
StartDate          :  1998-01-01
EndDate            :  2008-07-31
MaxTweets          :  -1
Exclude Retweets?  :  y
Exclude Replies?   :  y


In [34]:
def search(text,username,since,until,retweet,replies): 
    global filename 
    q = text 
    if username!='': 
        q += f" from:{username}"
    if until=='': 
        until = datetime.datetime.strftime(datetime.date.today(), '%Y-%m-%d') 
    q += f" until:{until}" 
    if since=='': 
        since = datetime.datetime.strftime(datetime.datetime.strptime(until, '%Y-%m-%d') -  datetime.timedelta(days=3650), '%Y-%m-%d') 
    q += f" since:{since}" 
    if retweet == 'y': 
        q += f" exclude:retweets"
    if replies == 'y': 
        q += f" exclude:replies"
    return q 

In [35]:
#Calling the Search Function and creating Dataframe

text = """South Africa (USDZAR OR (Rand AND Currency) OR (Exchange AND Rate))"""

q = search(text,username,since,until,retweet,replies) 
print(q)

# Creating list to append tweet data  
tweets_list1 = [] 

from datetime import datetime
start_time = datetime.now()

# Using TwitterSearchScraper to scrape data and append tweets to list 
if count == -1: 
    for i,tweet in enumerate(tqdm_notebook(sntwitter.TwitterSearchScraper(q, maxEmptyPages=100).get_items())): 
        tweets_list1.append([tweet.date, tweet.id, tweet.rawContent, tweet.user.username,tweet.lang,
                             tweet.replyCount,tweet.retweetCount, tweet.likeCount,tweet.quoteCount]) 

else: 
    with tqdm_notebook(total=count) as pbar: 
        for i,tweet in enumerate(sntwitter.TwitterSearchScraper(q, maxEmptyPages=100).get_items()): #declare a username  
            if i>=count: #number of tweets you want to scrape 
                break 
            tweets_list1.append([tweet.date, tweet.id, tweet.rawContent, tweet.user.username,tweet.lang,tweet.replyCount, 
                                    tweet.retweetCount,tweet.likeCount,tweet.quoteCount]) 
            pbar.update(1) 

# Creating a dataframe from the tweets list above  
tweets_df1 = pd.DataFrame(tweets_list1, columns=['DateTime', 'TweetId', 'Text', 'Username','Language', 
                                                 'ReplyCount','RetweetCount','LikeCount','QuoteCount']) 

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

South Africa (USDZAR OR (Rand AND Currency) OR (Exchange AND Rate)) until:2008-07-31 since:1998-01-01 exclude:retweets exclude:replies


0it [00:00, ?it/s]

Stopping after 100 empty pages


Duration: 0:00:37.767160


In [19]:
tweets_df1.sort_values(by='DateTime',ascending=False) 

,DateTime,TweetId,Text,Username,Language,ReplyCount,RetweetCount,LikeCount,QuoteCount
0,2023-06-27 20:46:16+00:00,1673794902180724736,"Pounds weakens vs Naira, see today's Dollar, E...",Nigeriansong1,pt,0,1,0,0
1,2023-06-27 03:30:57+00:00,1673534355342229505,See the Dollar to Naira exchange rate after Po...,Nigeriansong1,en,0,1,0,0
2,2023-06-24 06:29:22+00:00,1672492091807547397,Check out today's Dollar to Naira exchange rat...,Nigeriansong1,pt,0,0,1,0
3,2023-06-23 11:12:28+00:00,1672200948221976576,Check out today's Dollar to Naira exchange rat...,Nigeriansong1,pt,0,1,0,0
4,2023-06-23 11:05:05+00:00,1672199090250055685,Check out today's Dollar to Naira exchange rat...,Nigeriansong1,en,0,1,1,0
...,...,...,...,...,...,...,...,...,...
8538,2009-02-25 13:08:04+00:00,1249070957,South Africa's inflation figures are just out ...,daveharcourt,en,0,0,0,0
8539,2009-01-12 04:19:18+00:00,1112375857,Buying tickets in South Africa: Due to the ZAR...,mfeed,en,0,0,0,0
8540,2009-01-07 05:44:45+00:00,1101270778,JSE Securities Exchange South Africa Goes Live...,FSOkx,en,0,0,0,0
8541,2008-10-08 08:14:48+00:00,951012643,"In South Africa/Namibia, I have the rare privi...",CarrieCizauskas,en,0,0,0,0


In [20]:
#Data Processing
tweets_df1.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8543 entries, 0 to 8542
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   DateTime      8543 non-null   datetime64[ns, UTC]
 1   TweetId       8543 non-null   int64              
 2   Text          8543 non-null   object             
 3   Username      8543 non-null   object             
 4   Language      8543 non-null   object             
 5   ReplyCount    8543 non-null   int64              
 6   RetweetCount  8543 non-null   int64              
 7   LikeCount     8543 non-null   int64              
 8   QuoteCount    8543 non-null   int64              
dtypes: datetime64[ns, UTC](1), int64(5), object(3)
memory usage: 600.8+ KB


In [21]:
#Data Transformation
tweets_df1['Hour']       = tweets_df1['DateTime'].dt.hour 
tweets_df1['Year']       = tweets_df1['DateTime'].dt.year
tweets_df1['Month']      = tweets_df1['DateTime'].dt.month
tweets_df1['MonthName']  = tweets_df1['DateTime'].dt.month_name()
tweets_df1['MonthDay']   = tweets_df1['DateTime'].dt.day
tweets_df1['DayName']    = tweets_df1['DateTime'].dt.day_name()
tweets_df1['Week']       = tweets_df1['DateTime'].dt.isocalendar().week

In [22]:
#Split date and time into separate columns
tweets_df1['Date'] = [d.date() for d in tweets_df1['DateTime']] 
tweets_df1['Time'] = [d.time() for d in tweets_df1['DateTime']] 

In [23]:
#save to csv
tweets_df1.to_csv(f"Tweet_Data.csv",index=False)